In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import mdtraj as md
%matplotlib inline

In [ ]:
def rms(xvg):
    
    with open(xvg) as f:
        lines = f.readlines()
        drawparams=[ l for l in lines if l.startswith('@')]
        data=[[float(i) for i in l.split()] for l in lines if not l.startswith('#') and not l.startswith('@')]
    M=re.compile(r'.*\"(.*)\".*')
    title = re.match(M, [p for p in drawparams if p.split()[1] == 'title'][0]).group(1)
    xlabel = re.match(M, [p for p in drawparams if p.split()[1] == 'xaxis'][0]).group(1)
    ylabel = re.match(M, [p for p in drawparams if p.split()[1] == 'yaxis'][0]).group(1)
    #subtitle=re.match(M, [p for p in drawparams if p.split()[1] == 'subtitle'][0]).group(1)
    df=pd.DataFrame(data,columns=[xlabel,ylabel])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    #plt.suptitle(subtitle)
    plt.plot(df[xlabel],df[ylabel],label=title)
    plt.legend()
    plt.savefig('{}.png'.format(title),dpi=300)

In [ ]:
def output_seq(pdb_file):
    
    trans = {
    'ALA':'A','CYS':'C','ASP':'D','GLU':'E',
    'PHE':'F','GLY':'G','HIS':'H','LYS':'K',
    'ILE':'I','LEU':'L','MET':'M','ASN':'N',
    'PRO':'P','GLN':'Q','ARG':'R','SER':'S',
    'THR':'T','VAL':'V','TYR':'Y','TRP':'W'
    }
    with open(pdb_file) as f:
        lines = f.readlines()
        data=[[str(i) for i in l.split()] for l in lines if l.startswith('ATOM')]
    list=[]
    single=[]
    ss=[]
    df=pd.DataFrame(data)
    for i in df.index.values:
        if i < df.index.values[-1]:
            if df.iloc[i,:][5] != df.iloc[i+1,:][5]:         
                list.append(df.iloc[i,:][3])
                single.append(trans[df.iloc[i,:][3]])
                ss.append(trans[df.iloc[i,:][3]]+str(df.iloc[i,:][5]))
        else:
            list.append(df.iloc[i,:][3])
            single.append(trans[df.iloc[i,:][3]])
            ss.append(trans[df.iloc[i,:][3]]+str(df.iloc[i,:][5]))
    ex_name=pdb_file.split(".")[0]
    seq=pd.DataFrame(list,columns=["Amino acid"])
    seq['Single']=single
    seq['SS']=ss
    seq.to_excel('Seq_%s.xlsx' %ex_name)

In [ ]:
# 计算两个原子之间的距离
def distance(xtc,top,atom1,atom2,gap=1):
    traj=md.load_xtc(xtc,top=top,stride=gap)
    frames=traj.n_frames
    total_frame=gap*frames
    m=md.compute_distances(traj,atom_pairs=[[atom1,atom2]])
    df=pd.DataFrame(m,columns=['distances'],index=np.arange(0,total_frame,gap))
    plt.plot(df,label=df.columns.values[0])
    plt.xlabel('frames')
    plt.ylabel('distance')
    plt.xlim((0,total_frame))
    plt.title("Distance between Atom{} and Atom{} (nm)".format(atom1,atom2))
    plt.legend()
    plt.show()